# Part 1

In [1]:
def add_position(current_position, delta):
    return (current_position[0]+delta[0], current_position[1] +  delta[1])

In [2]:
def in_bound(pos, map_matrix):
    max_row = map_matrix.shape[0]
    max_column = map_matrix.shape[1]
    return pos[0] in range(max_row) and pos[1] in range(max_column)

In [3]:
def get_delta(move):
    if move == '>':
        return (0,1)
    elif move == '<':
        return (0,-1)
    elif move == '^':
        return (-1,0)
    else:
        return (1,0)

In [4]:
from datetime import datetime
import numpy as np
with open('input_day15.txt') as file:
    lines = file.readlines()
    lines = [line for line in lines]
    room_map = []
    movements = []
    for line in lines:
        if '#' in line:
            room_map.append([c for c in line.strip()])
        elif line != '\n':
            movements.extend([c for c in line.strip()])
    room_map = np.array(room_map)
    
    start_position = (0,0)
    for i in range(room_map.shape[0]):
        for j in range(room_map.shape[1]):
            if room_map[i,j] == '@':
                start_position = (i,j)
                break
                
    print(start_position)
    
    current_position = start_position
#     print(room_map)
    for move in movements:
#         print(move)
        delta = get_delta(move)
        next_position = add_position(current_position, delta)
        if in_bound(next_position, room_map) and room_map[next_position] != '#':
            if room_map[next_position] == '.':
#                 print(f'{move} - Moving @({current_position} to {next_position})')
                room_map[current_position] = '.'
                room_map[next_position] = '@'
                current_position = next_position
            elif room_map[next_position] == 'O':
                tmp = next_position
                while room_map[tmp] == 'O':
                    tmp = add_position(tmp, delta)
                if room_map[tmp] == '.':
#                     print(f'{move} - Moving @({current_position} to {next_position}), Moving O {next_position} to {tmp}')
                    room_map[tmp] = 'O'
                    room_map[current_position] = '.'
                    room_map[next_position] = '@'
                    current_position = next_position
#             print(room_map)
    
    counter = 0
    for i in range(room_map.shape[0]):
        for j in range(room_map.shape[1]):
            if room_map[i,j] == 'O':
                counter = counter + 100*i + j
    print('Answer:', counter)

(24, 24)
Answer: 1360570


# Part 2

In [27]:
def add_position(current_position, delta):
    return (current_position[0]+delta[0], current_position[1] +  delta[1])

def in_bound(pos, map_matrix):
    max_row = map_matrix.shape[0]
    max_column = map_matrix.shape[1]
    return pos[0] in range(max_row) and pos[1] in range(max_column)

def get_delta(move):
    if move == '>':
        return (0,1)
    elif move == '<':
        return (0,-1)
    elif move == '^':
        return (-1,0)
    else:
        return (1,0)

def get_larger_map(room_map):
    total_rows = []
    for i in range(room_map.shape[0]):
        single_row = []
        for j in range(room_map.shape[1]):
            if room_map[i,j] == '#':
                single_row.extend(['#','#'])
            elif room_map[i,j] == 'O':
                single_row.extend(['[',']'])
            elif room_map[i,j] == '.':
                single_row.extend(['.','.'])
            elif room_map[i,j] == '@':
                single_row.extend(['@','.'])
        total_rows.append(single_row)
    return np.array(total_rows)


from datetime import datetime
import numpy as np

with open('input_day15.txt') as file:
    lines = file.readlines()
    lines = [line for line in lines]
    room_map = []
    movements = []
    for line in lines:
        if '#' in line:
            room_map.append([c for c in line.strip()])
        elif line != '\n':
            movements.extend([c for c in line.strip()])
    room_map = np.array(room_map)

    extended_map = get_larger_map(room_map)
    [print("".join(i)) for i in extended_map]

    start_position = (0, 0)
    for i in range(extended_map.shape[0]):
        for j in range(extended_map.shape[1]):
            if extended_map[i, j] == '@':
                start_position = (i, j)
                break

    print(start_position)


def get_box_paired_positions(extended_map, next_position, direction):
    #     print(next_position, extended_map[next_position])
    if direction == '^' or direction == 'v':
        if extended_map[next_position] == '[':
            return [next_position, (next_position[0], next_position[1] + 1)]
        elif extended_map[next_position] == ']':
            return [(next_position[0], next_position[1] - 1), next_position]
        else:
            return []
    elif direction == '>' or direction == '<':
        return [next_position]


#     else return []

def get_boxes_paired_positions(extended_map, next_positions, move_direction):
    result = []
    for p in next_positions:
        new_elements = get_box_paired_positions(extended_map, p, move_direction)
        for e in new_elements:
            if e not in result:
                result.append(e)
        # result.extend(new_elements)
    return result


def can_push(levels, extended_map):
    for i in reversed(range(len(levels))):
        positions = levels[i]
        next_positions = [add_position(p, delta) for p in positions]
        for n in next_positions:
            if extended_map[n] == '#':
                return False
    return True


def get_next_positions(current_positions, delta, extended_map):
    return [add_position(p, delta) for p in current_positions]


def contain_box(positions, extended_map):
    for p in positions:
        if extended_map[p] == '[' or extended_map[p] == ']':
            return True
    return False


def build_box_levels(current_position, delta, extended_map, move_direction):
    levels = [[current_position]]
    next_position = add_position(current_position, delta)
    #     print(next_position)
    tmp = get_box_paired_positions(extended_map, next_position, move_direction)
    #     print(tmp)
    while contain_box(tmp, extended_map):
        # print('new level')
        levels.append(tmp)
        next_positions = get_next_positions(tmp, delta, extended_map)
        # print("Next positions", next_positions)
        paired_next_positions = get_boxes_paired_positions(extended_map, next_positions, move_direction)
        # print("paired position", paired_next_positions)
        tmp = paired_next_positions

    return levels


#         if extended_map[next_position] == '[' or extended_map[next_position] == ']'
#         box_positions = get_box_paired_positions(extended_map, next_position)
#         levels.append(box_positions)
def swap(first_positions, second_postions, extended_map):
    for i in range(len(first_positions)):
        tmp = extended_map[first_positions[i]]
        extended_map[first_positions[i]] = extended_map[second_postions[i]]
        extended_map[second_postions[i]] = tmp


def move_box_levels(levels, extended_map, delta):
    for level in reversed(levels):
        next_positions = [add_position(p, delta) for p in level]
        swap(level, next_positions, extended_map)



current_position = start_position
for move in movements:
    # print(current_position, move)
    # print('Before', move)
    # [print("".join(i)) for i in extended_map]
    delta = get_delta(move)
    #         next_position = add_position(current_position, delta)
    levels = build_box_levels(current_position, delta, extended_map, move)
    # print('Level:', levels)
    if len(levels) == 1:
        next_position = add_position(current_position, delta)
        if extended_map[next_position] == '.':
            extended_map[current_position] = '.'
            extended_map[next_position] = '@'
            current_position = next_position
        # elif extended_map[next_position] == '#':
            # print('Nothing move!')
    else:
        if can_push(levels, extended_map):
            move_box_levels(levels, extended_map, delta)
            next_position = add_position(current_position, delta)
            current_position = next_position
            # [print("".join(i)) for i in extended_map]
    # print('After:', move)
    # [print("".join(i)) for i in extended_map]
# print('Done')
counter = 0
for i in range(extended_map.shape[0]):
    for j in range(extended_map.shape[1]):
        if extended_map[i,j] == '[':
            counter = counter + 100*i + j
print('Answer:', counter)


####################################################################################################
##....[]..##..[]..........[]..##....[]......[]..[]..[]....[]....[]......[]......[]..........[][]..##
##........[][]................[]................[]....[][]..[]....[]......####[]..[]##[]....[][][]##
##....[]......[]............[]..[]##......[]..##............##[]..##............[]..........[][]..##
##[]..##[]........[][][][]..[][]........[]..............[]..##..[]....[]####......[]....##....##..##
##..[]....[]......[][]............[][][][]......................##[][]..[]....####..[]..[]..##....##
##..............[]..[]..[][]......##..[][][]....[]..[]....[]..[][]........[]......[]..........[][]##
##[]......[]........[]......[][]##..........[][]..[]....[]............##....##..[]..[][]..[]......##
##..[]..............[]..##..##....##..[]..............[][]....[]..........[]##..[][]..[]......[][]##
##[][]..##....##....##......[]..........[]..[]..........................[][]..[][][]..[]...